In [1]:
from dotenv import load_dotenv, find_dotenv 
from langchain_community.utilities import BingSearchAPIWrapper 
from langchain_community.tools.bing_search import BingSearchResults 
import json

load_dotenv(find_dotenv(), override = True)

True

In [2]:
search = BingSearchAPIWrapper()

tool = BingSearchResults(api_wrapper = search)  

In [3]:
def Bing_Search(query: str):
    response = tool.invoke(query)  
    response = json.loads(response.replace("'", '"')) 
    return response

In [4]:
output = Bing_Search("What is the current weather in Shanghai?") 

print(output)
print(output[0]["link"])

[{'snippet': 'World Asia China <b>Shanghai</b> <b>Shanghai</b>. <b>Shanghai</b>, <b>Shanghai</b>, China <b>Weather</b> Forecast, with <b>current</b> conditions, wind, air quality, and what to expect for the next 3 days.', 'title': 'Shanghai, Shanghai, China Weather Forecast | AccuWeather', 'link': 'https://www.accuweather.com/en/cn/shanghai/106577/weather-forecast/106577'}, {'snippet': '<b>Current weather</b> <b>in Shanghai</b>, <b>Shanghai</b>, China. Check <b>current</b> conditions <b>in Shanghai</b>, <b>Shanghai</b>, China with radar, hourly, and more.', 'title': 'Shanghai, Shanghai, China Current Weather | AccuWeather', 'link': 'https://www.accuweather.com/en/cn/shanghai/106577/current-weather/106577'}, {'snippet': '<b>Current</b> <b>weather</b> <b>in Shanghai</b> and forecast for today, tomorrow, and next 14 days', 'title': 'Weather in Shanghai, Shanghai Municipality, China - timeanddate.com', 'link': 'https://www.timeanddate.com/weather/china/shanghai'}, {'snippet': 'Observed at 

In [4]:
def Bing_Search_Link_Wrapper(func): 
    def wrapped_function(query): 
        response = func(query) 
        list_of_links = [response[i]["link"] for i in range(len(response))]
        return list_of_links
    return wrapped_function

In [5]:
@Bing_Search_Link_Wrapper
def Bing_Search_Wrapped(query: str): 
    response = tool.invoke(query)  
    response = json.loads(response.replace("'", '"')) 
    return response

In [7]:
output_2 = Bing_Search_Wrapped(query = "What is the current weather in Shanghai?")

In [8]:
print(output_2)
for i in output_2: 
    print("-" * 50) 
    print(i) 

['https://www.accuweather.com/en/cn/shanghai/106577/weather-forecast/106577', 'https://www.accuweather.com/en/cn/shanghai/106577/current-weather/106577', 'https://www.timeanddate.com/weather/china/shanghai', 'https://www.bbc.com/weather/1796236']
--------------------------------------------------
https://www.accuweather.com/en/cn/shanghai/106577/weather-forecast/106577
--------------------------------------------------
https://www.accuweather.com/en/cn/shanghai/106577/current-weather/106577
--------------------------------------------------
https://www.timeanddate.com/weather/china/shanghai
--------------------------------------------------
https://www.bbc.com/weather/1796236


### Integrating Apify with Bing Search

In [6]:
from langchain.indexes import VectorstoreIndexCreator 
from langchain_community.utilities import ApifyWrapper 
from langchain_core.documents import Document
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override = True)

True

In [7]:
llm = ChatOpenAI(model_name = "gpt-4o-mini", temperature = 0)

In [8]:
apify = ApifyWrapper()

embedding = OpenAIEmbeddings(model = "text-embedding-3-small")

/tmp/ipykernel_566561/3508749125.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embedding = OpenAIEmbeddings(model = "text-embedding-3-small")


In [9]:
def apify_search(link: list, query: str): 
    loader = apify.call_actor(
        actor_id = "apify/website-content-crawler", 
        run_input = {"startUrls": [{"url": link[0]}]}, 
        dataset_mapping_function = lambda item: Document(
            page_content = item["text"] or "", metadata = {"source": item["url"]}
        )
    )
    
    index = VectorstoreIndexCreator(embedding = embedding).from_loaders([loader]) 
    
    output = index.query_with_sources(llm = llm, question = query)
    
    return output

In [10]:
def Apify_Search_Wrapper(func_2): 
    def wrapper_function(query: str): 
        link = func_2(query)[0]
        loader = apify.call_actor(
            actor_id = "apify/website-content-crawler", 
            run_input = {"startUrls": [{"url": link}]}, 
            dataset_mapping_function = lambda item: Document(
                page_content = item["text"] or "", metadata = {"source": item["url"]}
            )
        )
        
        index = VectorstoreIndexCreator(embedding = embedding).from_loaders([loader]) 
        
        output = index.query_with_sources(llm = llm, question = query)
        
        return output
    return wrapper_function

In [11]:
def Bing_Search_Link_Wrapper(func): 
    def wrapped_function(query): 
        response = func(query) 
        list_of_links = [response[i]["link"] for i in range(len(response))]
        return list_of_links
    return wrapped_function

In [12]:
@Apify_Search_Wrapper
@Bing_Search_Link_Wrapper
def Bing_Search_Wrapped_Twice(query: str): 
    response = tool.invoke(query)  
    response = json.loads(response.replace("'", '"')) 
    return response

In [21]:
output_wrapped_twice = Bing_Search_Wrapped_Twice(query = "What is the score of the Spurs vs Arsenal premier league game on September 15, 2024?")

/home/mpariaszevski/anaconda3/envs/AutoGPT_Project/lib/python3.11/site-packages/langchain/indexes/vectorstore.py:128: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(


In [22]:
print(output_wrapped_twice)

{'question': 'What is the score of the Spurs vs Arsenal premier league game on September 15, 2024?', 'answer': 'Arsenal won the match against Tottenham with a final score of 1-0 on September 15, 2024.  \n', 'sources': 'https://www.espn.com/football/match/_/gameId/704317/arsenal-tottenham-hotspur'}


In [13]:
output_wrapped_twice_2 = Bing_Search_Wrapped_Twice(query = "What were the opinions from fans of the Arsenal and Spurs game from September 15, 2024?")

/home/mpariaszevski/anaconda3/envs/AutoGPT_Project/lib/python3.11/site-packages/langchain/indexes/vectorstore.py:128: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(


In [14]:
print(output_wrapped_twice_2["answer"])

I don't know.  

